In [0]:
# Imports
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
DATA_DIR   = "/content/drive/My Drive/sample_10/"
IMG_WIDTH  = 512
IMG_HEIGHT = 512
TEST_SIZE  = 0.15

In [3]:
images_list = list()

for file in os.listdir(DATA_DIR):
  images_list.append(file)

print(images_list)

['Places365_test_00000156.jpg', 'Places365_test_00000149.jpg', 'Places365_test_00000148.jpg', 'Places365_test_00000176.jpg', 'Places365_test_00000144.jpg', 'Places365_test_00000179.jpg', 'Places365_test_00000157.jpg', 'Places365_test_00000162.jpg', 'Places365_test_00000158.jpg', 'Places365_test_00000164.jpg']


In [0]:
# Find color array index
# Interval == 20
def calculate_color_index_interval_20(a,b):
  if (a == 250):
    a = 240
  if (b == 250):
    b = 240
  index = round((a / 20), 0) * 13 + round((b / 20), 0)
  return int(index)


# Round Function
def round_numpy_array(numpy_array):
  new_rounded_list = []

  for row in numpy_array:
    new_row = []
    for element in row:
      new_row.append(round(element, -1))
    new_rounded_list.append(new_row)
  
  return new_rounded_list


# Round and calculate (For Interval of 20)
def round_and_calculate_interval_20(a_list, b_list):
  rounded_a_list = round_numpy_array(a_list.tolist())
  rounded_b_list = round_numpy_array(b_list.tolist())
  index_array = []

  for i in range(0, len(rounded_a_list)):
    new_row = []
    for j in range(0, len(rounded_a_list[i])):
      index = calculate_color_index_interval_20(rounded_a_list[i][j], rounded_b_list[i][j])
      new_row.append(index)
    index_array.append(new_row)
  
  return index_array


# For Interval 20
def process_image_interval_20(image_path, width, height):
  img = cv2.imread(image_path) # read image
  img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB) # convert from BGR to LAB
  resized_img = cv2.resize(img_lab, (width,height)) # resize image

  # split into L, a and b
  L_array = resized_img[:, :, 0]
  a_array = resized_img[:, :, 1]
  b_array = resized_img[:, :, 2]

  target_array = round_and_calculate_interval_20(a_array, b_array)

  # NumPy data type needs to be 'dtype=np.uint16' or higher
  numpy_target_array = np.array(target_array, dtype=np.uint16)

  return (L_array, numpy_target_array)

In [0]:
def prepare_data(images_list, width, height):

  x_list = list()
  y_list = list()

  for image in images_list:
    image_path = DATA_DIR + image
    x, y = process_image_interval_20(image_path, width, height)
    x_BGR = cv2.cvtColor(x, cv2.COLOR_GRAY2BGR) # cloned 1 channel to 3 channel for model input
    x_list.append(x_BGR)
    y_list.append(y)

  return x_list, y_list


def split_data_train_test(x_list, y_list, test_size):
  
  x_train, x_test, y_train, y_test = train_test_split(x_list, y_list, test_size=test_size)
  
  np.save("/content/drive/My Drive/sample_10/x_train.npy", x_train)
  np.save("/content/drive/My Drive/sample_10/y_train.npy", y_train)
  np.save("/content/drive/My Drive/sample_10/x_test.npy", x_test)
  np.save("/content/drive/My Drive/sample_10/y_test.npy", y_test)

In [0]:
x_list, y_list = prepare_data(images_list, IMG_WIDTH, IMG_HEIGHT)

In [0]:
split_data_train_test(x_list, y_list, TEST_SIZE)

In [0]:
x_train = np.load("/content/drive/My Drive/sample_10/x_train.npy", allow_pickle=True)
y_train = np.load("/content/drive/My Drive/sample_10/y_train.npy", allow_pickle=True)
x_test = np.load("/content/drive/My Drive/sample_10/x_test.npy", allow_pickle=True)
y_test = np.load("/content/drive/My Drive/sample_10/y_test.npy", allow_pickle=True)

In [14]:
print(x_train[0])

[[[ 73  73  73]
  [ 73  73  73]
  [ 74  74  74]
  ...
  [120 120 120]
  [121 121 121]
  [119 119 119]]

 [[ 75  75  75]
  [ 75  75  75]
  [ 77  77  77]
  ...
  [121 121 121]
  [121 121 121]
  [119 119 119]]

 [[ 77  77  77]
  [ 77  77  77]
  [ 77  77  77]
  ...
  [122 122 122]
  [121 121 121]
  [120 120 120]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
